### St Michael Bay

**Station Name:**  at01

**Location:**  St Michael, Alaska

**Archive:**  [UNAVCO](http://www.unavco.org)

**Ellipsoidal Coordinates:**

- Latitude:  63.4840

- Longitude: -162.0064

- Height: 21.7 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/AT01)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/AT01.sta)

[Google Map Link](https://goo.gl/maps/uWyoNFf4DRjYLmfUA)

<p align="center">
<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/AT01.jpg" width="500">
</p>

## Data Summary

at01 has good visibility over a large azimuthal region and the monument is about a meter taller than normal geodetic monuments which helps improve visibility of the sea surface. At01 observes all constellation signals.

## Imports

In [ ]:
import json
import os
import sys

import seaborn as sns;

sns.set_theme(style="whitegrid");

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)

import gnssrefl_helpers

#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment(refl_code="../../..", orbits="../../../orbits", exe="../../../bin/exe")

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.make_json_input import make_json
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.subdaily_cl import subdaily
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

In [ ]:
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

# reflecion zone app

now we can use the reflection zone part of the web app:

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/geoid" width="1000" height="600"></iframe>

Next, we could look at the reflection zone app to pick a possible mask.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones" width="1000" height="700"></iframe>

Just looking at this can give us an idea of the possible azimuth mask - Guessing we will not get good retrievals in the northwest quadrant. We will confirm this with the qc plots when we use quicklook.

It's also useful to note that it is about 12 meters above sea level.

### Take a Quick Look at the Data
First examine the spectral characteristics of the SNR data.

First, we download a snr file using rinex2snr. We will pick a year and day - in this case we'll look at day of year 109 for the year 2020. 

In [ ]:
# Download SNR 
# we are going to just pick a year and day
# this will generate a SNR file at a sampling rate of 15 seconds.
station = 'at01'
year=2020
doy=109

rinex2snr(station=station, year=year, doy=doy, archive='unavco', orb='gnss')

Now we can run quicklook, this is what it will look like if we don't change any of the defualts:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

Using the default settings with quicklook we would likely think it is not a useful site, as nearly every retrieval is set as bad.

Let's see what the defaults are set to by adding a question mark to the function and running that cell.

In [ ]:
quicklook?

Now we can see what the defaults are and information about the parameters in the doc string as well. 

So, remembering that the site is 12 meters above sea level and we can see that the default restricts the reflector height region to 0.5 to 6 meters. 

Let's try quicklook again using a reflector height region that includes the water surface. We'll set h1 to 8 and h2 to 15 and we're also going to change the elevation angles to 5 and 13 degrees:

In [ ]:
values, metrics = quicklook(station, year, doy=doy,
                                         e1=5, e2=13, h1=8, h2=15)

Now we see good retrievals at azimuths from true north to about 220 degrees (around what we expected from using the reflection zone app) and we also see strong retrievals in the Lomb Scargle periodograms.

This site does have modern GPS signals, as well as Galileo and Glonass signals.

results for L2C:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, 
                                         e1=5, e2=13, h1=8, h2=15, fr=20)

Now we can look at glonass by setting fr to 101 and Galileo we will set f to 205.

In [ ]:
values_l1, metrics_l1 = quicklook(station, year, doy=doy,
                                         e1=5, e2=13, h1=8, h2=15, fr=101)

In [ ]:
values_l2, metrics_l2 = quicklook(station, year, doy=doy,
                                         e1=5, e2=13, h1=8, h2=15, fr=205)

### Analyze the Data

Next we will analyze data for a few weeks in the fall of 2020. Our first step would be to make the SNR files.

In [ ]:
rinex2snr?

In [ ]:
rinex2snr(station, year, 230, doy_end=251, archive='unavco', orb='gnss')

The next step is to create the json file that gnssir reads with the values that we have decided on from the quicklook step. In this case we will set the elevation angles and the heights. We will also set the parameter allfreq to true.

In [ ]:
lat = 63.484
long = -162.006
height = 21.565

make_json(station, lat, long, height, e1=5, e2=13, h1=8, h2=15, allfreq=True)

In [ ]:
# This is the json file that was created with the defaults/parameters you set above
json_file = f'{refl_code_loc}/input/{station}.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    
file

To set the azimuth range that we decided on, we need to edit the json file manually. We are also going to edit out the Beidou signals since they are not in rinex2 files.

In [ ]:
# Now lets edit the json file
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    
# Here is where we can 'hand edit' values in the json file
# lets edit the azimuths. We set these values by looking at the metrics qc plot above
file['azval'] = [20,90,90,180,180,220]
file['freqs'] = [1,20,5,101,102,201,205,206,207]
os.remove(json_file)

with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
# now lets view it again and note the difference
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Next, we will run gnssir to estimate the reflector height (RH) for the two month time period.

In [ ]:
year = 2020
doy = 230
doy_end = 251
gnssir(station, year, doy, doy_end=doy_end, screenstats=False)

We have do have some code to help you look at subdaily files. This will be helpful to look at tides. 

We will set an outlier criteria of 0.36 meters (one sigma 0.12 meters ==> 0.36 meters three sigma) to start with.

This preliminary version of the code removes outliers and makes an effort to compute the RH dot correction.

In [ ]:
subdaily(station='at01', year=2020, doy1=230, doy2=251, plt=True)